In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Download and load the pre-trained BERT model and tokenizer from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')


C:\Users\prana\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
import nltk

# NLTK movie dataset
nltk.download('movie_reviews')
nltk.download('punkt')

def load_movie_reviews():
    reviews = []
    labels = []
    for fileid in movie_reviews.fileids():
        # Use the first letter of the filename as the label: 'pos' or 'neg'
        label = 1 if fileid.split('/')[0] == 'pos' else 0
        with open(movie_reviews.abspath(fileid), 'r') as f:
            reviews.append(f.read())
            labels.append(label)
    return reviews, labels

reviews, labels = load_movie_reviews()

# training and test sets
X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Convert labels to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Tokenization and padding
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 500
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

# Build a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(
    X_train_pad, y_train,
    epochs=3,
    batch_size=32,
    validation_data=(X_test_pad, y_test)
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_acc}")


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/3
50/50 [==============================] - 1s 13ms/step - loss: 0.6899 - accuracy: 0.5506 - val_loss: 0.6825 - val_accuracy: 0.5475
Epoch 2/3
50/50 [==============================] - 1s 14ms/step - loss: 0.6428 - accuracy: 0.7094 - val_loss: 0.6078 - val_accuracy: 0.7525
Epoch 3/3
13/13 [==============================] - 0s 2ms/step - loss: 0.4788 - accuracy: 0.7725
Test Accuracy: 0.7724999785423279


In [10]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling1D

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# additional dense layers
input_ids = Input(shape=(512,), dtype='int32')
attention_mask = Input(shape=(512,), dtype='int32')

# Extract BERT outputs
bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

pooled_output = GlobalAveragePooling1D()(bert_output)

# Add dense layers on top of BERT output
dense_output = Dense(64, activation='relu')(pooled_output)
output = Dense(1, activation='sigmoid')(dense_output)

modified_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# different learning rate
modified_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), loss='binary_crossentropy', metrics=['accuracy'])

def encode_data(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors='tf')
    return encodings['input_ids'], encodings['attention_mask']

X_train_enc = encode_data(X_train)
X_test_enc = encode_data(X_test)

# Train
modified_model.fit(
    [X_train_enc[0], X_train_enc[1]],  # input_ids and attention_mask
    y_train,
    epochs=3,
    batch_size=32,
    validation_data=([X_test_enc[0], X_test_enc[1]], y_test)
)

# Evaluate
modified_results = modified_model.evaluate([X_test_enc[0], X_test_enc[1]], y_test)
print(f"Modified Test Accuracy: {modified_results[1]}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
50/50 [==============================] - 3215s 64s/step - loss: 0.6009 - accuracy: 0.6419 - val_loss: 0.4507 - val_accuracy: 0.7950
Epoch 2/3
50/50 [==============================] - 1758s 35s/step - loss: 0.3253 - accuracy: 0.8656 - val_loss: 0.3968 - val_accuracy: 0.8150
Epoch 3/3
13/13 [==============================] - 134s 10s/step - loss: 0.3043 - accuracy: 0.8850
Modified Test Accuracy: 0.8849999904632568
